# <center> <img src='../images/fsktm.jpg' width="500" height="400"> </center>
# <center> WQD7002 - SCIENCE DATA RESEARCH PROJECT </center>
## <center> NEURAL RUBIK’S </center>
## <center> Solving Rubik's Cube Using Nueral Network (Hueristic Learning) </center>
### <center> Scripted by : Gunasegarran Magadevan (WQD170002) </center>
### <center> Supervised by : Dr.Aznul Qalid Md Sabri </center>
# <center> <img src="../images/RubiksNeural.jpg" width="400" height="300"> </center>
----



# STEP 1 : Installing and upgrading the package

In [2]:
# Upgrading the pip package to the latest version
!python -m pip install --upgrade pip --quiet
!python -m pip install PyHamcrest --quiet
!python -m pip install tensorflow --quiet


#Kindly run tensorflow package manually through terminal or cmd - https://anaconda.org/conda-forge
# conda install -c conda-forge tensorflow
# conda install -c conda-forge numpy

# Installing cubeai - https://pypi.org/project/cubeai
## Library for Use Machine Learning to learn a heuristic
!python -m pip uninstall cubeai -y --quiet
!python -m pip install cubeai --no-cache-dir --upgrade --quiet


# Installing seaborn - https://pypi.org/project/cubeai
## Library for making statistical graphics in Python.
!python -m pip uninstall seaborn -y --quiet
!python -m pip install seaborn --no-cache-dir --upgrade --quiet

# Installing Keras - https://pypi.org/project/Keras
## Library capable of running on top of TensorFlow, CNTK, or Theano.
!python -m pip uninstall keras -y --quiet
!python -m pip install keras --no-cache-dir --upgrade --quiet

### STEP 2 : Importing the package

In [3]:
# Importing packages
import cubeai as cai                # Cube Modules.
import numpy as np                  # To manipulate large multi-dimensional arrays .
import pandas as pd                 # To use data structures and data analysis tools.
import seaborn as sns               # To create statistical graphics.
import matplotlib.pyplot as plt     # To create 2D graphics.
import keras                        # High-level neural networks.

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### STEP 3 : Configurations & Functions

In [4]:
# Configurating for graph plotting - https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_style("whitegrid")
sns.set_context("talk")
sns.set_palette("deep")
plt.rcParams.update({'figure.figsize': (10, 8)})

In [5]:
# save/load model functions
def save_model(model, filename):
  global SAVE_TO_DRIVE
  if SAVE_TO_DRIVE is True:
    model.save(filename)
    upload_file_to_drive(filename, filename)
    return


def load_model(filename):
  return download_model_from_drive(filename, filename)

### STEP 4 : Training the module

In [6]:
def get_features_from_cube(cube):
  """ transforms the cube's array to 1d binary array """
  binary_array = keras.utils.to_categorical(cube.to_array(), cai.NUM_FACES)
  return binary_array.flatten()

 
def data_generator(cube_layers, max_d, batch_size, p=None):
  """
  generates batches of scrambled cubes data, coupled with the number
  of scramble moves per row
  """
  new_dim = len(get_features_from_cube(cai.Cube(cube_layers)))
  while True:
    data = np.empty((batch_size, new_dim), dtype=np.int8)
    labels = np.empty(batch_size, dtype=np.int8)
    for i in range(batch_size):
      c = cai.Cube(cube_layers)
      d = np.random.choice(np.arange(max_d+1), p=p)
      rand_seq = cai.generate_random_sequence(cube_layers, d)
      c.apply(rand_seq)
      data[i, :] = get_features_from_cube(c)
      labels[i] = d
    yield data, labels  


def create_dnnregressor(cube_layers, hidden_units, dropout=None,
                        optimizer='adagrad', loss='mse'):
  """
  creates a fully connected multi-layer perceptron with non-linear activations
  to perform regression.
  
  :param cube_layers: the number of cube layers this model should operate on
  :param hidden_units: list of integers specifying how many hidden neurons
                       are in each layer
  :param dropout: if None, no dropout is used. if a single integer, uses this
                  dropout rate after each layer. otherwise, should be a list of
                  integers the same length as hidden_units specifying dropout 
                  rate after each layer
  :param optimizer: which (keras) optimizer to use
  :param loss: which (keras) loss to use
  :returns: a compiled keras.Sequential model
  """
  # input checks
  assert hasattr(hidden_units, '__len__'), 'hidden_units must be array-like'
  assert len(hidden_units) > 0, 'hidden_units cannot be empty'
  if dropout is not None:
    if not hasattr(dropout, '__len__'):
      dropout = [dropout] * len(hidden_units)
    else:
      assert len(hidden_units) == len(dropout)
  # define some constant model parameters
  activation = 'relu'
  out_activation = 'relu'
  input_dim = len(get_features_from_cube(cai.Cube(cube_layers)))
  
  # create a sequential model and add the first layer
  model = keras.Sequential()
  model.add(keras.layers.Dense(hidden_units[0],
                               input_dim=input_dim,
                               activation=activation))
  if dropout is not None:
    model.add(keras.layers.Dropout(dropout[0]))
    
  # add the rest of the hidden layers
  for i in range(1, len(hidden_units)):
    model.add(keras.layers.Dense(hidden_units[i], activation=activation))
    if dropout is not None:
      model.add(keras.layers.Dropout(dropout[i]))

  # define the output layer
  model.add(keras.layers.Dense(1, activation=out_activation))
  model.compile(optimizer=optimizer, loss=loss)
  return model


def learn_heuristic(layers, max_d, p, steps, epochs, batch_size, hidden_units,
                    dropout=None, optimizer='adagrad', loss='mse'):
  """
  trains a model with the given config.
  
  :param layers: number of cube layers
  :param max_d: maximum number of scramble steps
  :param p: a probability distribution according to which the
            scramble steps number is chosen (array of length max_d+1)
            (for uniform dist. use None)
  :param steps: number of training steps per epoch
  :param epochs: number of epochs
  :param batch_size: number of cube instances in each training step
  :param hidden_units: number of dnn layers and neurons in each layer
                       (an array of integers)
  :param dropout: same as in create_dnnregressor
  :param optimizer: same as in create_dnnregressor
  :param loss: same as in create_dnnregressor
  :returns: a pair (estimator, history), where estimator is a (trained)
            keras model, and history is the training Keras history object
  """
  # set up parameters
  c = cai.Cube(layers)
  # initialize the model
  estimator = create_dnnregressor(
      cube_layers=layers,
      hidden_units=hidden_units,
      dropout=dropout,
      optimizer=optimizer,
      loss=loss
  )
  # train the model
  history = estimator.fit_generator(
      data_generator(layers, max_d, batch_size, p), steps, epochs
  )
  return estimator, history


def model_to_heuristic(model):
  """ creates a heuristic based on the given keras model """
  
  def _model_h(cube, problem=None):
    features = get_features_from_cube(cube)
    return model.predict(np.reshape(features, (1, -1)))[0][0]
  
  return _model_h

#### $3\times3\times3$
We try different network architectures and see which one performs best.

The heuristics are then save in the following format:

`<layers>_<max_d>_<hidden_1>_..._<hidden_k>.h5`

where `hidden_i` is the number of neurons in the `i`'th hidden layer, and `layers` is the number of layers in the cube, and `max_d` is the maximal number of scramble moves.

For example, a $3\times 3 \times 3$ model with 3 layers of 50 neurons each, and `max_d=8` is saved as: 
"`3_8_50_50_50.h5`".